<a href="https://colab.research.google.com/github/jorgegarcia396/hacking-civico/blob/master/Accidentes_viales_CDMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statistics

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
av = '/content/drive/My Drive/incidentes-viales-c5.csv'

In [4]:
data_av = pd.read_csv(av, encoding='latin-1')

In [5]:
data_av.columns = data_av.columns.str.replace(' ', '_')

In [6]:
data_av.head()

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,aÃ±o_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
0,IZ/200101/04110,01/01/2020,08:49:57,MiÃ©rcoles,(A) La unidad de atenciÃ³n a emergencias fue d...,01/01/2020,2020,Enero,10:26:26,TLAHUAC,accidente-motociclista,19.30303,-99.05604,URGENCIAS MEDICAS,RADIO,TLAHUAC,"19.30302999,-99.05604012",1
1,C5/200101/00743,01/01/2020,01:04:38,MiÃ©rcoles,(A) La unidad de atenciÃ³n a emergencias fue d...,01/01/2020,2020,Enero,01:18:36,AZCAPOTZALCO,accidente-choque sin lesionados,19.49422,-99.19734,EMERGENCIA,LLAMADA DEL 911,AZCAPOTZALCO,"19.49421996,-99.19734012",1
2,C5/200101/00012,31/12/2019,23:59:58,Martes,(A) La unidad de atenciÃ³n a emergencias fue d...,01/01/2020,2020,Enero,06:59:44,IZTAPALAPA,accidente-choque sin lesionados,19.32178,-99.05313,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,"19.32177996,-99.05312988",1
3,GA/191231/08094,31/12/2019,23:46:07,Martes,(A) La unidad de atenciÃ³n a emergencias fue d...,01/01/2020,2020,Enero,00:22:00,VENUSTIANO CARRANZA,accidente-motociclista,19.45154,-99.09475,URGENCIAS MEDICAS,RADIO,VENUSTIANO CARRANZA,"19.45153998,-99.09474984",1
4,C5/200101/03024,01/01/2020,05:26:06,MiÃ©rcoles,(N) La unidad de atenciÃ³n a emergencias fue d...,01/01/2020,2020,Enero,06:55:23,MAGDALENA CONTRERAS,accidente-choque sin lesionados,19.32809,-99.21489,EMERGENCIA,LLAMADA DEL 911,MAGDALENA CONTRERAS,"19.32809004,-99.21489012",1


In [7]:
accidentes_viales = data_av.drop(['folio','dia_semana', 'codigo_cierre', 'fecha_cierre', 'aÃ±o_cierre', 'mes_cierre', 'hora_cierre', 'latitud', 'longitud', 'clas_con_f_alarma', 'tipo_entrada', 'delegacion_cierre', 'mes'], axis=1)
accidentes_viales.head()

,fecha_creacion,hora_creacion,delegacion_inicio,incidente_c4,geopoint
0,01/01/2020,08:49:57,TLAHUAC,accidente-motociclista,"19.30302999,-99.05604012"
1,01/01/2020,01:04:38,AZCAPOTZALCO,accidente-choque sin lesionados,"19.49421996,-99.19734012"
2,31/12/2019,23:59:58,IZTAPALAPA,accidente-choque sin lesionados,"19.32177996,-99.05312988"
3,31/12/2019,23:46:07,VENUSTIANO CARRANZA,accidente-motociclista,"19.45153998,-99.09474984"
4,01/01/2020,05:26:06,MAGDALENA CONTRERAS,accidente-choque sin lesionados,"19.32809004,-99.21489012"


In [8]:
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha_creacion" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

accidentes_viales = cast_datetime_cols(accidentes_viales)

In [9]:
accidentes_viales['mes'] = accidentes_viales.fecha_creacion.dt.month
accidentes_viales['año'] = accidentes_viales.fecha_creacion.dt.year

In [10]:
numerical = accidentes_viales.select_dtypes(include='number')
categorical = accidentes_viales.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

# **Exploración de datos**

¿Cuáles son las alcaldías con mayor número de accidentes?

In [11]:
accidentes_viales.delegacion_inicio.value_counts()

IZTAPALAPA             18724
GUSTAVO A. MADERO      10858
CUAUHTEMOC             10341
TLALPAN                 8130
COYOACAN                7438
VENUSTIANO CARRANZA     7422
BENITO JUAREZ           7415
ALVARO OBREGON          7132
MIGUEL HIDALGO          7108
XOCHIMILCO              4804
AZCAPOTZALCO            4627
IZTACALCO               4412
TLAHUAC                 2965
CUAJIMALPA              1874
MAGDALENA CONTRERAS     1759
MILPA ALTA               591
Name: delegacion_inicio, dtype: int64

¿Cuáles son los tipos de accidentes con mayor número de registros?

In [12]:
accidentes_viales.incidente_c4.value_counts()

accidente-choque sin lesionados                    49071
accidente-choque con lesionados                    28666
lesionado-atropellado                              11953
accidente-motociclista                              9969
accidente-volcadura                                 2769
accidente-ciclista                                  1417
accidente-persona atrapada / desbarrancada           500
accidente-vehÃ­culo atrapado-varado                  450
accidente-choque con prensados                       246
cadÃ¡ver-accidente automovilÃ­stico                  113
cadÃ¡ver-atropellado                                 109
sismo-choque con lesionados                          101
detenciÃ³n ciudadana-atropellado                      80
accidente-vehiculo desbarrancado                      63
accidente-otros                                       53
detenciÃ³n ciudadana-accidente automovilÃ­stico       27
accidente-ferroviario                                 10
sismo-persona atropellada      

In [13]:
accidentes_viales.hora_creacion.value_counts()

17:09:00    11
19:54:00    11
17:50:09    10
16:26:00    10
11:35:00    10
            ..
21:08:14     1
05:45:30     1
10:03:25     1
16:13:50     1
20:39:50     1
Name: hora_creacion, Length: 56712, dtype: int64

Accidentes automovilísticos por mes...

In [14]:
accidentes_viales.groupby(['mes', 'delegacion_inicio']).count()

fecha_creacion  hora_creacion  ...  geopoint   año
mes delegacion_inicio                                   ...                
1   ALVARO OBREGON                 1094           1094  ...      1094  1094
    AZCAPOTZALCO                    698            698  ...       698   698
    BENITO JUAREZ                   984            984  ...       984   984
    COYOACAN                       1128           1128  ...      1128  1128
    CUAJIMALPA                      230            230  ...       230   230
...                                 ...            ...  ...       ...   ...
12  MILPA ALTA                       27             27  ...        27    27
    TLAHUAC                         113            113  ...       113   113
    TLALPAN                         276            276  ...       276   276
    VENUSTIANO CARRANZA             244            244  ...       244   244
    XOCHIMILCO                      198            198  ...       198   198

[192 rows x 5 columns]

In [15]:
import altair as alt
import seaborn as sns
from datetime import datetime

In [16]:
accidentes_viales['fecha'] = accidentes_viales['fecha_creacion'].astype(str)
accidentes_viales['hora'] = accidentes_viales['hora_creacion'].astype(str)


In [17]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
alt.Chart(accidentes_viales).mark_bar().encode(
    x = alt.X("delegacion_inicio:N"),
    y = "count()"
)

In [ ]:
alt.Chart(accidentes_viales).mark_circle().encode(
    x = alt.X('incidente_c4'),
    y = 'count()',
    color = alt.Color('delegacion_inicio', title = 'Delegación'),
).properties(
    title = "Accidentes por alcaldía"
)

## Conclusiones

Como se puede observar en las gráficas anteriores, la alcaldía de Iztapalapa en la Ciudad de México registra el mayor número de incidentes viales, clasificados bajos las siguientes cuatro tipologías mostradas por número de registros:

*   "Choque sin lesionados"
*   "Choque con lesionados"
*   "Lesionado atroppelado"
*   "Accidente motociclista"

Pese a que este hecho llama la atención, es importante destacar que los incidentes con mayor número de registros no generan lesionados y esto puede deberse a que en dicha alcaldía el número de vías de acceso controlado (vías principales) no es tan elevado como sí lo es en alcaldías como: Miguel Hidalgo, Cuauhtémoc, Benito Juárez y Coyoacán.

Es importante mencionar que, precismanete en el caso de vías de acceso controlado, la Secretaría de Movilidad de la Ciudad de México estableció que los vehículos o cualquier otro transporte puede circular a una velocidad de 80km/h y en el caso de ejes viales y avenidas principales, la velocidad permitida es de 50km/h.

También se puede notar que los accidentes con mayor incidencia, ocurren a en su mayoría a las 19:54 horas, por lo que quizá una solución que podría implementar el gobierno de la Ciudad de México sería instalar puestos de control de velocidad en las zonas más conflictivas para que de esta forma se pudieran controlar o disminuir el número de incidentes viales.


La información utilizada en este proyecto puede ser consultada en el siguiente link:
https://datos.cdmx.gob.mx/explore/dataset/incidentes-viales-c5/table/?disjunctive.incidente_c4&location=10,19.33685,-99.15797